# Helper functions (this chunk can be ignored)

In [173]:
# !pip install xmltodict
# !pip install plotly
# !pip install pandas
# !pip install numpy

In [174]:
# Import the required modules

import xmltodict
import pandas as pd
import numpy as np
import plotly.io as pio
import collections
import plotly.express as px
import numpy as np
pio.renderers.default='notebook'
pd.set_option('display.max_rows', 300)


In [3]:

def xml_to_dict(
    filename: str
)-> dict:
    # Open the file and read the contents
    with open(filename, 'rb') as file:
        my_xml = file.read()
        file.close()

    my_dict = xmltodict.parse(my_xml)
    return my_dict

In [4]:

def deep_convert_dict(
    layer: dict
) -> dict:
    to_ret = layer
    if isinstance(layer, collections.OrderedDict):
        to_ret = dict(layer)

    try:
        for key, value in to_ret.items():
            to_ret[key] = deep_convert_dict(value)
    except AttributeError:
        pass

    return to_ret

# IMPORTANT: Add the path to the elisa and flisa xml files
- this should be the only part that requires user input
- elisa use the reduced file
- flisa use the raw file

In [37]:
!pwd

/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer


In [114]:
runfolder = "/Users/keithmitchell/Downloads/"
#runfolder = "/Users/keithmitchell/Desktop/Repositories/TrimmerLabGeneral/lab_processing/new_run6/"
#elisa = runfolder + "COS-ELISA_2023-16-03_Jph-3_N499_reduced_xml.xml"
elisa = runfolder + "2L_9_one_reduced.xml"
#flisa = runfolder + "Protein_FLISA_2023-16-03_Jph-3_N499_raw_xml.xml"
flisa = runfolder + "2L_9_one_raw.xml"

# ELISA

In [115]:
master_list = []
for i in range(0,len(xml_to_dict(filename=elisa)["Experiment"]["PlateSections"])):
    get_just_reduced = deep_convert_dict(xml_to_dict(filename=elisa)["Experiment"]["PlateSections"][i])
    get_just_reduced = get_just_reduced["PlateSection"]["reducedData"]["Well"]
    for w in get_just_reduced:
        w = deep_convert_dict(w)
        w["plate"] = i+1
        master_list.append(w)


In [116]:
df_e = pd.DataFrame.from_records(master_list)
df_e

,@WellID,@Name,reducedVal,plate
0,1,A1,0.2762,1
1,2,A2,0.1241,1
2,3,A3,0.1742,1
3,4,A4,0.2296,1
4,5,A5,0.1451,1
...,...,...,...,...
3451,92,H8,0.2639,36
3452,93,H9,0.3182,36
3453,94,H10,0.2363,36
3454,95,H11,0.2739,36


# FLISA
- note we need the raw for this input

In [117]:
import pandas as pd

master_list = []
start_dict = xml_to_dict(filename=flisa)["Experiment"]["PlateSections"]
for i in range(0,len(start_dict)):
    get_just_reduced = deep_convert_dict(start_dict[i])
    get_just_reduced = get_just_reduced["PlateSection"]
    get_just_reduced = deep_convert_dict(get_just_reduced["Wavelengths"]["Wavelength"])
    if type(deep_convert_dict(get_just_reduced)) == list:
        for wv in range(0,len(get_just_reduced)):
            grab_wavelength = deep_convert_dict(get_just_reduced[wv])["@WavelengthIndex"]
            get_just_reduced_wv = deep_convert_dict(get_just_reduced[wv])
            get_just_reduced_wells = [deep_convert_dict(i) for i in get_just_reduced_wv["Wells"]["Well"]]
            for w in get_just_reduced_wells:
                w["wavelength"] = grab_wavelength
                w["plate"] = i+1
                master_list.append(w)
    else:
        grab_wavelength = deep_convert_dict(get_just_reduced)["@WavelengthIndex"]
        get_just_reduced_wv = deep_convert_dict(get_just_reduced)
        get_just_reduced_wells = [deep_convert_dict(i) for i in get_just_reduced_wv["Wells"]["Well"]]
        for w in get_just_reduced_wells:
            w["wavelength"] = grab_wavelength
            w["plate"] = i+1
            master_list.append(w)
            
            

In [152]:
df_f = pd.DataFrame.from_records(master_list)
df_f

,@WellID,@Name,@Row,@Col,RawData,wavelength,plate
0,1,A1,1,1,0.2762,1,1
1,2,A2,1,2,0.1241,1,1
2,3,A3,1,3,0.1742,1,1
3,4,A4,1,4,0.2296,1,1
4,5,A5,1,5,0.1451,1,1
...,...,...,...,...,...,...,...
3451,92,H8,8,8,0.2639,1,36
3452,93,H9,8,9,0.3182,1,36
3453,94,H10,8,10,0.2363,1,36
3454,95,H11,8,11,0.2739,1,36


In [153]:
df_f.index = df_f["@Name"] + "_Plate" + df_f["plate"].astype(str)
df_f = df_f[["RawData", "wavelength"]].pivot(columns='wavelength').droplevel(0, axis=1) 
df_f["1"] = df_f["1"].astype(float)+1

In [154]:
df_f

wavelength,1
A10_Plate1,1.1195
A10_Plate10,1.2407
A10_Plate11,1.2551
A10_Plate12,1.3954
A10_Plate13,1.2466
...,...
H9_Plate5,1.2232
H9_Plate6,1.2528
H9_Plate7,1.3373
H9_Plate8,1.3454


# Normalize to well after merging ELISA and FLISA

In [155]:
df_e.index = df_e["@Name"] + "_Plate" + df_e["plate"].astype(str) # add the plate to help with normalization process
final_df = df_e.merge(df_f, left_index=True, right_index=True)

In [156]:
final_df

,@WellID,@Name,reducedVal,plate,1
A1_Plate1,1,A1,0.2762,1,1.2762
A2_Plate1,2,A2,0.1241,1,1.1241
A3_Plate1,3,A3,0.1742,1,1.1742
A4_Plate1,4,A4,0.2296,1,1.2296
A5_Plate1,5,A5,0.1451,1,1.1451
...,...,...,...,...,...
H8_Plate36,92,H8,0.2639,36,1.2639
H9_Plate36,93,H9,0.3182,36,1.3182
H10_Plate36,94,H10,0.2363,36,1.2363
H11_Plate36,95,H11,0.2739,36,1.2739


In [157]:
master_norm_list = []
for i in set(final_df["plate"].to_list()):
    # filter for the plate
    cols_in = [i for i in ["reducedVal", "1","2","3"] if i in final_df.columns]
    temp_fil = final_df[final_df["plate"]==i][cols_in]
    
    # fix the column types
    for col in temp_fil.columns:
        temp_fil[col] = temp_fil[col].astype(float)
        # lets log scale this as well
        temp_fil[col] = np.log2(temp_fil[col]+1)

    # get the positive control and divide by it
    pos_control1 = temp_fil.loc["A1_Plate" + str(i)]
    pos_control2 = temp_fil.loc["A12_Plate" + str(i)]
    pos_control = (pos_control1 + pos_control2)/2
    #temp_fil = temp_fil/pos_control

    # column sum normalize
    for col in temp_fil.columns:
        temp_fil[col] = temp_fil[col]/temp_fil[col].sum()

    #pos_control = temp_fil.loc[["A1_Plate" + str(i), "A12_Plate" + str(i)]]

    master_norm_list.append(temp_fil)

final_df = pd.concat(master_norm_list)
    

In [158]:
final_df.columns = ["ELISA", "FLISAwv1", "FLISAwv2", "FLISAwv3"][:len(final_df.columns)]
final_df["Control"] = ""
for index, row in final_df.iterrows():
    if "A12_" in index or "A1_" in index:
        final_df.at[index, "Control"] = "Positive Control"
    if "H1_" in index or "H12_" in index:
        final_df.at[index, "Control"] = "Negative Control"
final_df["Name"] = final_df.index

In [159]:
final_df

,ELISA,FLISAwv1,Control,Name
A1_Plate1,0.016042,0.011042,Positive Control,A1_Plate1
A2_Plate1,0.007695,0.010114,,A2_Plate1
A3_Plate1,0.010563,0.010427,,A3_Plate1
A4_Plate1,0.013595,0.010765,,A4_Plate1
A5_Plate1,0.008912,0.010246,,A5_Plate1
...,...,...,...,...
H8_Plate36,0.010376,0.010404,,H8_Plate36
H9_Plate36,0.012240,0.010706,,H9_Plate36
H10_Plate36,0.009398,0.010248,,H10_Plate36
H11_Plate36,0.010725,0.010460,,H11_Plate36


In [160]:
final_df["FLISA_wv1 + ELISA"] = final_df["FLISAwv1"] + final_df["ELISA"]
if "FLISAwv2" in final_df.columns:
    # calculate sum for max later
    final_df["FLISA_wv2 + ELISA"] = final_df["FLISAwv2"] + final_df["ELISA"]
if "FLISAwv3" in final_df.columns:
    final_df["FLISA_wv3 + ELISA"] = final_df["FLISAwv3"] + final_df["ELISA"]
flisa_cols = [i for i in final_df.columns if "FLISA" in i]
final_df["FLISA"] = final_df[[i for i in final_df.columns if "FLISA" in i]].sum(axis=1)
final_df["Max"] = final_df[[i for i in final_df.columns if "FLISA" in i]].idxmax(axis=1)

### Final Normalized and Annotated Dataframe

In [161]:
final_df.to_csv(runfolder + "final_df.csv")
final_df

,ELISA,FLISAwv1,Control,Name,FLISA_wv1 + ELISA,FLISA,Max
A1_Plate1,0.016042,0.011042,Positive Control,A1_Plate1,0.027084,0.038127,FLISA
A2_Plate1,0.007695,0.010114,,A2_Plate1,0.017809,0.027922,FLISA
A3_Plate1,0.010563,0.010427,,A3_Plate1,0.020990,0.031416,FLISA
A4_Plate1,0.013595,0.010765,,A4_Plate1,0.024360,0.035125,FLISA
A5_Plate1,0.008912,0.010246,,A5_Plate1,0.019158,0.029404,FLISA
...,...,...,...,...,...,...,...
H8_Plate36,0.010376,0.010404,,H8_Plate36,0.020781,0.031185,FLISA
H9_Plate36,0.012240,0.010706,,H9_Plate36,0.022946,0.033652,FLISA
H10_Plate36,0.009398,0.010248,,H10_Plate36,0.019646,0.029894,FLISA
H11_Plate36,0.010725,0.010460,,H11_Plate36,0.021186,0.031646,FLISA


# IMPORTANT: Visuals and beginning of actual reporting of interest

- 1: Wavelength 1 vs ELISA and top 50
- 2: Wavelength 2 vs ELISA and top 50
- 3: Wavelength 3 vs ELISA and top 50

## 1: Wavelength 1 vs ELISA and top 50

In [162]:
import plotly.express as px
fig = px.scatter(final_df, x="ELISA", y="FLISAwv1", color="Control", symbol = "Max", hover_data=["Name",] + flisa_cols)
fig.show()

In [163]:
final_df.sort_values("FLISA_wv1 + ELISA",ascending=False)[:300]

,ELISA,FLISAwv1,Control,Name,FLISA_wv1 + ELISA,FLISA,Max
A12_Plate26,0.033783,0.013575,Positive Control,A12_Plate26,0.047358,0.060933,FLISA
A2_Plate26,0.031009,0.013172,,A2_Plate26,0.044182,0.057354,FLISA
A12_Plate33,0.029961,0.013252,Positive Control,A12_Plate33,0.043213,0.056465,FLISA
A12_Plate34,0.029748,0.013245,Positive Control,A12_Plate34,0.042992,0.056237,FLISA
A12_Plate35,0.029200,0.013202,Positive Control,A12_Plate35,0.042402,0.055604,FLISA
F7_Plate25,0.028292,0.013405,,F7_Plate25,0.041697,0.055101,FLISA
H7_Plate33,0.028621,0.013043,,H7_Plate33,0.041664,0.054707,FLISA
A12_Plate22,0.028124,0.013358,Positive Control,A12_Plate22,0.041482,0.054840,FLISA
E11_Plate35,0.028016,0.013014,,E11_Plate35,0.041030,0.054044,FLISA
A12_Plate32,0.027852,0.012967,Positive Control,A12_Plate32,0.040819,0.053786,FLISA


## 2: Wavelength 2 vs ELISA and top 50

In [164]:
if "FLISAwv2" in flisa_cols:
    import plotly.express as px
    fig = px.scatter(final_df, x="ELISA", y="FLISAwv2", color="Control", symbol = "Max", hover_data=["Name", "FLISAwv1", "FLISAwv3"])
    fig.show()

In [165]:
if "FLISAwv2" in flisa_cols:
    final_df.sort_values("FLISA_wv2 + ELISA",ascending=False)[:50] # 

## 3: Wavelength 3 vs ELISA and the top 50

In [166]:
if "FLISAwv3" in flisa_cols:
    import plotly.express as px
    fig = px.scatter(final_df, x="ELISA", y="FLISAwv3", color="Control", symbol = "Max", hover_data=["Name", "FLISAwv1", "FLISAwv2"])
    fig.show()

In [167]:
if "FLISAwv3" in flisa_cols:
    final_df.sort_values("FLISA_wv3 + ELISA",ascending=False)[:50]

# 4: Wavelength FLISA Max vs ELISA

In [168]:
import plotly.express as px
fig = px.scatter(final_df, x="ELISA", y="FLISA", color="Control", symbol = "Max", hover_data=["Name",]+ flisa_cols)
fig.show()

In [169]:
final_df.sort_values("FLISA",ascending=False)[:100]

,ELISA,FLISAwv1,Control,Name,FLISA_wv1 + ELISA,FLISA,Max
A12_Plate26,0.033783,0.013575,Positive Control,A12_Plate26,0.047358,0.060933,FLISA
A2_Plate26,0.031009,0.013172,,A2_Plate26,0.044182,0.057354,FLISA
A12_Plate33,0.029961,0.013252,Positive Control,A12_Plate33,0.043213,0.056465,FLISA
A12_Plate34,0.029748,0.013245,Positive Control,A12_Plate34,0.042992,0.056237,FLISA
A12_Plate35,0.029200,0.013202,Positive Control,A12_Plate35,0.042402,0.055604,FLISA
F7_Plate25,0.028292,0.013405,,F7_Plate25,0.041697,0.055101,FLISA
A12_Plate22,0.028124,0.013358,Positive Control,A12_Plate22,0.041482,0.054840,FLISA
H7_Plate33,0.028621,0.013043,,H7_Plate33,0.041664,0.054707,FLISA
H5_Plate29,0.026938,0.013687,,H5_Plate29,0.040625,0.054313,FLISA
E11_Plate35,0.028016,0.013014,,E11_Plate35,0.041030,0.054044,FLISA


# Which samples are high in all three (show up in the top 300 in all three lists?)

In [170]:
if "FLISAwv2" in flisa_cols and "FLISAwv3" in flisa_cols:
    filter_indexes = list((set(final_df.sort_values("FLISAwv1",ascending=False)[:300].index) & 
    set(final_df.sort_values("FLISAwv2",ascending=False)[:300].index) &
    set(final_df.sort_values("FLISAwv3",ascending=False)[:300].index)))
    final_df.loc[filter_indexes][:50]

# XYZ 3d graph for the 3 FLISA wavelengths

In [171]:
if "FLISAwv2" in flisa_cols and "FLISAwv3" in flisa_cols:
    final_df["ELISA_pow"] = final_df["ELISA"]+10
    fig = px.scatter_3d(final_df, x="FLISAwv1", y="FLISAwv2", z="FLISAwv3",
                  color="Control", size="ELISA_pow")
    fig.show()

# Ignore from here down

In [175]:
%%js
/** vim: et:ts=4:sw=4:sts=4
 * @license RequireJS 2.3.6 Copyright jQuery Foundation and other contributors.
 * Released under MIT license, https://github.com/requirejs/requirejs/blob/master/LICENSE
 */
var requirejs,require,define;!function(global,setTimeout){var req,s,head,baseElement,dataMain,src,interactiveScript,currentlyAddingScript,mainScript,subPath,version="2.3.6",commentRegExp=/\/\*[\s\S]*?\*\/|([^:"'=]|^)\/\/.*$/gm,cjsRequireRegExp=/[^.]\s*require\s*\(\s*["']([^'"\s]+)["']\s*\)/g,jsSuffixRegExp=/\.js$/,currDirRegExp=/^\.\//,op=Object.prototype,ostring=op.toString,hasOwn=op.hasOwnProperty,isBrowser=!("undefined"==typeof window||"undefined"==typeof navigator||!window.document),isWebWorker=!isBrowser&&"undefined"!=typeof importScripts,readyRegExp=isBrowser&&"PLAYSTATION 3"===navigator.platform?/^complete$/:/^(complete|loaded)$/,defContextName="_",isOpera="undefined"!=typeof opera&&"[object Opera]"===opera.toString(),contexts={},cfg={},globalDefQueue=[],useInteractive=!1;function commentReplace(e,t){return t||""}function isFunction(e){return"[object Function]"===ostring.call(e)}function isArray(e){return"[object Array]"===ostring.call(e)}function each(e,t){var i;if(e)for(i=0;i<e.length&&(!e[i]||!t(e[i],i,e));i+=1);}function eachReverse(e,t){var i;if(e)for(i=e.length-1;-1<i&&(!e[i]||!t(e[i],i,e));i-=1);}function hasProp(e,t){return hasOwn.call(e,t)}function getOwn(e,t){return hasProp(e,t)&&e[t]}function eachProp(e,t){var i;for(i in e)if(hasProp(e,i)&&t(e[i],i))break}function mixin(i,e,r,n){return e&&eachProp(e,function(e,t){!r&&hasProp(i,t)||(!n||"object"!=typeof e||!e||isArray(e)||isFunction(e)||e instanceof RegExp?i[t]=e:(i[t]||(i[t]={}),mixin(i[t],e,r,n)))}),i}function bind(e,t){return function(){return t.apply(e,arguments)}}function scripts(){return document.getElementsByTagName("script")}function defaultOnError(e){throw e}function getGlobal(e){if(!e)return e;var t=global;return each(e.split("."),function(e){t=t[e]}),t}function makeError(e,t,i,r){var n=new Error(t+"\nhttps://requirejs.org/docs/errors.html#"+e);return n.requireType=e,n.requireModules=r,i&&(n.originalError=i),n}if(void 0===define){if(void 0!==requirejs){if(isFunction(requirejs))return;cfg=requirejs,requirejs=void 0}void 0===require||isFunction(require)||(cfg=require,require=void 0),req=requirejs=function(e,t,i,r){var n,o,a=defContextName;return isArray(e)||"string"==typeof e||(o=e,isArray(t)?(e=t,t=i,i=r):e=[]),o&&o.context&&(a=o.context),(n=getOwn(contexts,a))||(n=contexts[a]=req.s.newContext(a)),o&&n.configure(o),n.require(e,t,i)},req.config=function(e){return req(e)},req.nextTick=void 0!==setTimeout?function(e){setTimeout(e,4)}:function(e){e()},require||(require=req),req.version=version,req.jsExtRegExp=/^\/|:|\?|\.js$/,req.isBrowser=isBrowser,s=req.s={contexts:contexts,newContext:newContext},req({}),each(["toUrl","undef","defined","specified"],function(t){req[t]=function(){var e=contexts[defContextName];return e.require[t].apply(e,arguments)}}),isBrowser&&(head=s.head=document.getElementsByTagName("head")[0],baseElement=document.getElementsByTagName("base")[0],baseElement&&(head=s.head=baseElement.parentNode)),req.onError=defaultOnError,req.createNode=function(e,t,i){var r=e.xhtml?document.createElementNS("http://www.w3.org/1999/xhtml","html:script"):document.createElement("script");return r.type=e.scriptType||"text/javascript",r.charset="utf-8",r.async=!0,r},req.load=function(t,i,r){var e,n=t&&t.config||{};if(isBrowser)return(e=req.createNode(n,i,r)).setAttribute("data-requirecontext",t.contextName),e.setAttribute("data-requiremodule",i),!e.attachEvent||e.attachEvent.toString&&e.attachEvent.toString().indexOf("[native code")<0||isOpera?(e.addEventListener("load",t.onScriptLoad,!1),e.addEventListener("error",t.onScriptError,!1)):(useInteractive=!0,e.attachEvent("onreadystatechange",t.onScriptLoad)),e.src=r,n.onNodeCreated&&n.onNodeCreated(e,n,i,r),currentlyAddingScript=e,baseElement?head.insertBefore(e,baseElement):head.appendChild(e),currentlyAddingScript=null,e;if(isWebWorker)try{setTimeout(function(){},0),importScripts(r),t.completeLoad(i)}catch(e){t.onError(makeError("importscripts","importScripts failed for "+i+" at "+r,e,[i]))}},isBrowser&&!cfg.skipDataMain&&eachReverse(scripts(),function(e){if(head||(head=e.parentNode),dataMain=e.getAttribute("data-main"))return mainScript=dataMain,cfg.baseUrl||-1!==mainScript.indexOf("!")||(mainScript=(src=mainScript.split("/")).pop(),subPath=src.length?src.join("/")+"/":"./",cfg.baseUrl=subPath),mainScript=mainScript.replace(jsSuffixRegExp,""),req.jsExtRegExp.test(mainScript)&&(mainScript=dataMain),cfg.deps=cfg.deps?cfg.deps.concat(mainScript):[mainScript],!0}),define=function(e,i,t){var r,n;"string"!=typeof e&&(t=i,i=e,e=null),isArray(i)||(t=i,i=null),!i&&isFunction(t)&&(i=[],t.length&&(t.toString().replace(commentRegExp,commentReplace).replace(cjsRequireRegExp,function(e,t){i.push(t)}),i=(1===t.length?["require"]:["require","exports","module"]).concat(i))),useInteractive&&(r=currentlyAddingScript||getInteractiveScript())&&(e||(e=r.getAttribute("data-requiremodule")),n=contexts[r.getAttribute("data-requirecontext")]),n?(n.defQueue.push([e,i,t]),n.defQueueMap[e]=!0):globalDefQueue.push([e,i,t])},define.amd={jQuery:!0},req.exec=function(text){return eval(text)},req(cfg)}function newContext(u){var i,e,l,c,d,g={waitSeconds:7,baseUrl:"./",paths:{},bundles:{},pkgs:{},shim:{},config:{}},p={},f={},r={},h=[],m={},n={},v={},x=1,b=1;function q(e,t,i){var r,n,o,a,s,u,c,d,p,f,l=t&&t.split("/"),h=g.map,m=h&&h["*"];if(e&&(u=(e=e.split("/")).length-1,g.nodeIdCompat&&jsSuffixRegExp.test(e[u])&&(e[u]=e[u].replace(jsSuffixRegExp,"")),"."===e[0].charAt(0)&&l&&(e=l.slice(0,l.length-1).concat(e)),function(e){var t,i;for(t=0;t<e.length;t++)if("."===(i=e[t]))e.splice(t,1),t-=1;else if(".."===i){if(0===t||1===t&&".."===e[2]||".."===e[t-1])continue;0<t&&(e.splice(t-1,2),t-=2)}}(e),e=e.join("/")),i&&h&&(l||m)){e:for(o=(n=e.split("/")).length;0<o;o-=1){if(s=n.slice(0,o).join("/"),l)for(a=l.length;0<a;a-=1)if((r=getOwn(h,l.slice(0,a).join("/")))&&(r=getOwn(r,s))){c=r,d=o;break e}!p&&m&&getOwn(m,s)&&(p=getOwn(m,s),f=o)}!c&&p&&(c=p,d=f),c&&(n.splice(0,d,c),e=n.join("/"))}return getOwn(g.pkgs,e)||e}function E(t){isBrowser&&each(scripts(),function(e){if(e.getAttribute("data-requiremodule")===t&&e.getAttribute("data-requirecontext")===l.contextName)return e.parentNode.removeChild(e),!0})}function w(e){var t=getOwn(g.paths,e);if(t&&isArray(t)&&1<t.length)return t.shift(),l.require.undef(e),l.makeRequire(null,{skipMap:!0})([e]),!0}function y(e){var t,i=e?e.indexOf("!"):-1;return-1<i&&(t=e.substring(0,i),e=e.substring(i+1,e.length)),[t,e]}function S(e,t,i,r){var n,o,a,s,u=null,c=t?t.name:null,d=e,p=!0,f="";return e||(p=!1,e="_@r"+(x+=1)),u=(s=y(e))[0],e=s[1],u&&(u=q(u,c,r),o=getOwn(m,u)),e&&(u?f=i?e:o&&o.normalize?o.normalize(e,function(e){return q(e,c,r)}):-1===e.indexOf("!")?q(e,c,r):e:(u=(s=y(f=q(e,c,r)))[0],f=s[1],i=!0,n=l.nameToUrl(f))),{prefix:u,name:f,parentMap:t,unnormalized:!!(a=!u||o||i?"":"_unnormalized"+(b+=1)),url:n,originalName:d,isDefine:p,id:(u?u+"!"+f:f)+a}}function k(e){var t=e.id,i=getOwn(p,t);return i||(i=p[t]=new l.Module(e)),i}function M(e,t,i){var r=e.id,n=getOwn(p,r);!hasProp(m,r)||n&&!n.defineEmitComplete?(n=k(e)).error&&"error"===t?i(n.error):n.on(t,i):"defined"===t&&i(m[r])}function O(i,e){var t=i.requireModules,r=!1;e?e(i):(each(t,function(e){var t=getOwn(p,e);t&&(t.error=i,t.events.error&&(r=!0,t.emit("error",i)))}),r||req.onError(i))}function j(){globalDefQueue.length&&(each(globalDefQueue,function(e){var t=e[0];"string"==typeof t&&(l.defQueueMap[t]=!0),h.push(e)}),globalDefQueue=[])}function P(e){delete p[e],delete f[e]}function R(){var e,r,t=1e3*g.waitSeconds,n=t&&l.startTime+t<(new Date).getTime(),o=[],a=[],s=!1,u=!0;if(!i){if(i=!0,eachProp(f,function(e){var t=e.map,i=t.id;if(e.enabled&&(t.isDefine||a.push(e),!e.error))if(!e.inited&&n)w(i)?s=r=!0:(o.push(i),E(i));else if(!e.inited&&e.fetched&&t.isDefine&&(s=!0,!t.prefix))return u=!1}),n&&o.length)return(e=makeError("timeout","Load timeout for modules: "+o,null,o)).contextName=l.contextName,O(e);u&&each(a,function(e){!function n(o,a,s){var e=o.map.id;o.error?o.emit("error",o.error):(a[e]=!0,each(o.depMaps,function(e,t){var i=e.id,r=getOwn(p,i);!r||o.depMatched[t]||s[i]||(getOwn(a,i)?(o.defineDep(t,m[i]),o.check()):n(r,a,s))}),s[e]=!0)}(e,{},{})}),n&&!r||!s||!isBrowser&&!isWebWorker||d||(d=setTimeout(function(){d=0,R()},50)),i=!1}}function a(e){hasProp(m,e[0])||k(S(e[0],null,!0)).init(e[1],e[2])}function o(e,t,i,r){e.detachEvent&&!isOpera?r&&e.detachEvent(r,t):e.removeEventListener(i,t,!1)}function s(e){var t=e.currentTarget||e.srcElement;return o(t,l.onScriptLoad,"load","onreadystatechange"),o(t,l.onScriptError,"error"),{node:t,id:t&&t.getAttribute("data-requiremodule")}}function T(){var e;for(j();h.length;){if(null===(e=h.shift())[0])return O(makeError("mismatch","Mismatched anonymous define() module: "+e[e.length-1]));a(e)}l.defQueueMap={}}return c={require:function(e){return e.require?e.require:e.require=l.makeRequire(e.map)},exports:function(e){if(e.usingExports=!0,e.map.isDefine)return e.exports?m[e.map.id]=e.exports:e.exports=m[e.map.id]={}},module:function(e){return e.module?e.module:e.module={id:e.map.id,uri:e.map.url,config:function(){return getOwn(g.config,e.map.id)||{}},exports:e.exports||(e.exports={})}}},(e=function(e){this.events=getOwn(r,e.id)||{},this.map=e,this.shim=getOwn(g.shim,e.id),this.depExports=[],this.depMaps=[],this.depMatched=[],this.pluginMaps={},this.depCount=0}).prototype={init:function(e,t,i,r){r=r||{},this.inited||(this.factory=t,i?this.on("error",i):this.events.error&&(i=bind(this,function(e){this.emit("error",e)})),this.depMaps=e&&e.slice(0),this.errback=i,this.inited=!0,this.ignore=r.ignore,r.enabled||this.enabled?this.enable():this.check())},defineDep:function(e,t){this.depMatched[e]||(this.depMatched[e]=!0,this.depCount-=1,this.depExports[e]=t)},fetch:function(){if(!this.fetched){this.fetched=!0,l.startTime=(new Date).getTime();var e=this.map;if(!this.shim)return e.prefix?this.callPlugin():this.load();l.makeRequire(this.map,{enableBuildCallback:!0})(this.shim.deps||[],bind(this,function(){return e.prefix?this.callPlugin():this.load()}))}},load:function(){var e=this.map.url;n[e]||(n[e]=!0,l.load(this.map.id,e))},check:function(){if(this.enabled&&!this.enabling){var t,e,i=this.map.id,r=this.depExports,n=this.exports,o=this.factory;if(this.inited){if(this.error)this.emit("error",this.error);else if(!this.defining){if(this.defining=!0,this.depCount<1&&!this.defined){if(isFunction(o)){if(this.events.error&&this.map.isDefine||req.onError!==defaultOnError)try{n=l.execCb(i,o,r,n)}catch(e){t=e}else n=l.execCb(i,o,r,n);if(this.map.isDefine&&void 0===n&&((e=this.module)?n=e.exports:this.usingExports&&(n=this.exports)),t)return t.requireMap=this.map,t.requireModules=this.map.isDefine?[this.map.id]:null,t.requireType=this.map.isDefine?"define":"require",O(this.error=t)}else n=o;if(this.exports=n,this.map.isDefine&&!this.ignore&&(m[i]=n,req.onResourceLoad)){var a=[];each(this.depMaps,function(e){a.push(e.normalizedMap||e)}),req.onResourceLoad(l,this.map,a)}P(i),this.defined=!0}this.defining=!1,this.defined&&!this.defineEmitted&&(this.defineEmitted=!0,this.emit("defined",this.exports),this.defineEmitComplete=!0)}}else hasProp(l.defQueueMap,i)||this.fetch()}},callPlugin:function(){var u=this.map,c=u.id,e=S(u.prefix);this.depMaps.push(e),M(e,"defined",bind(this,function(e){var o,t,i,r=getOwn(v,this.map.id),n=this.map.name,a=this.map.parentMap?this.map.parentMap.name:null,s=l.makeRequire(u.parentMap,{enableBuildCallback:!0});return this.map.unnormalized?(e.normalize&&(n=e.normalize(n,function(e){return q(e,a,!0)})||""),M(t=S(u.prefix+"!"+n,this.map.parentMap,!0),"defined",bind(this,function(e){this.map.normalizedMap=t,this.init([],function(){return e},null,{enabled:!0,ignore:!0})})),void((i=getOwn(p,t.id))&&(this.depMaps.push(t),this.events.error&&i.on("error",bind(this,function(e){this.emit("error",e)})),i.enable()))):r?(this.map.url=l.nameToUrl(r),void this.load()):((o=bind(this,function(e){this.init([],function(){return e},null,{enabled:!0})})).error=bind(this,function(e){this.inited=!0,(this.error=e).requireModules=[c],eachProp(p,function(e){0===e.map.id.indexOf(c+"_unnormalized")&&P(e.map.id)}),O(e)}),o.fromText=bind(this,function(e,t){var i=u.name,r=S(i),n=useInteractive;t&&(e=t),n&&(useInteractive=!1),k(r),hasProp(g.config,c)&&(g.config[i]=g.config[c]);try{req.exec(e)}catch(e){return O(makeError("fromtexteval","fromText eval for "+c+" failed: "+e,e,[c]))}n&&(useInteractive=!0),this.depMaps.push(r),l.completeLoad(i),s([i],o)}),void e.load(u.name,s,o,g))})),l.enable(e,this),this.pluginMaps[e.id]=e},enable:function(){(f[this.map.id]=this).enabled=!0,this.enabling=!0,each(this.depMaps,bind(this,function(e,t){var i,r,n;if("string"==typeof e){if(e=S(e,this.map.isDefine?this.map:this.map.parentMap,!1,!this.skipMap),this.depMaps[t]=e,n=getOwn(c,e.id))return void(this.depExports[t]=n(this));this.depCount+=1,M(e,"defined",bind(this,function(e){this.undefed||(this.defineDep(t,e),this.check())})),this.errback?M(e,"error",bind(this,this.errback)):this.events.error&&M(e,"error",bind(this,function(e){this.emit("error",e)}))}i=e.id,r=p[i],hasProp(c,i)||!r||r.enabled||l.enable(e,this)})),eachProp(this.pluginMaps,bind(this,function(e){var t=getOwn(p,e.id);t&&!t.enabled&&l.enable(e,this)})),this.enabling=!1,this.check()},on:function(e,t){var i=this.events[e];i||(i=this.events[e]=[]),i.push(t)},emit:function(e,t){each(this.events[e],function(e){e(t)}),"error"===e&&delete this.events[e]}},(l={config:g,contextName:u,registry:p,defined:m,urlFetched:n,defQueue:h,defQueueMap:{},Module:e,makeModuleMap:S,nextTick:req.nextTick,onError:O,configure:function(e){if(e.baseUrl&&"/"!==e.baseUrl.charAt(e.baseUrl.length-1)&&(e.baseUrl+="/"),"string"==typeof e.urlArgs){var i=e.urlArgs;e.urlArgs=function(e,t){return(-1===t.indexOf("?")?"?":"&")+i}}var r=g.shim,n={paths:!0,bundles:!0,config:!0,map:!0};eachProp(e,function(e,t){n[t]?(g[t]||(g[t]={}),mixin(g[t],e,!0,!0)):g[t]=e}),e.bundles&&eachProp(e.bundles,function(e,t){each(e,function(e){e!==t&&(v[e]=t)})}),e.shim&&(eachProp(e.shim,function(e,t){isArray(e)&&(e={deps:e}),!e.exports&&!e.init||e.exportsFn||(e.exportsFn=l.makeShimExports(e)),r[t]=e}),g.shim=r),e.packages&&each(e.packages,function(e){var t;t=(e="string"==typeof e?{name:e}:e).name,e.location&&(g.paths[t]=e.location),g.pkgs[t]=e.name+"/"+(e.main||"main").replace(currDirRegExp,"").replace(jsSuffixRegExp,"")}),eachProp(p,function(e,t){e.inited||e.map.unnormalized||(e.map=S(t,null,!0))}),(e.deps||e.callback)&&l.require(e.deps||[],e.callback)},makeShimExports:function(t){return function(){var e;return t.init&&(e=t.init.apply(global,arguments)),e||t.exports&&getGlobal(t.exports)}},makeRequire:function(o,a){function s(e,t,i){var r,n;return a.enableBuildCallback&&t&&isFunction(t)&&(t.__requireJsBuild=!0),"string"==typeof e?isFunction(t)?O(makeError("requireargs","Invalid require call"),i):o&&hasProp(c,e)?c[e](p[o.id]):req.get?req.get(l,e,o,s):(r=S(e,o,!1,!0).id,hasProp(m,r)?m[r]:O(makeError("notloaded",'Module name "'+r+'" has not been loaded yet for context: '+u+(o?"":". Use require([])")))):(T(),l.nextTick(function(){T(),(n=k(S(null,o))).skipMap=a.skipMap,n.init(e,t,i,{enabled:!0}),R()}),s)}return a=a||{},mixin(s,{isBrowser:isBrowser,toUrl:function(e){var t,i=e.lastIndexOf("."),r=e.split("/")[0];return-1!==i&&(!("."===r||".."===r)||1<i)&&(t=e.substring(i,e.length),e=e.substring(0,i)),l.nameToUrl(q(e,o&&o.id,!0),t,!0)},defined:function(e){return hasProp(m,S(e,o,!1,!0).id)},specified:function(e){return e=S(e,o,!1,!0).id,hasProp(m,e)||hasProp(p,e)}}),o||(s.undef=function(i){j();var e=S(i,o,!0),t=getOwn(p,i);t.undefed=!0,E(i),delete m[i],delete n[e.url],delete r[i],eachReverse(h,function(e,t){e[0]===i&&h.splice(t,1)}),delete l.defQueueMap[i],t&&(t.events.defined&&(r[i]=t.events),P(i))}),s},enable:function(e){getOwn(p,e.id)&&k(e).enable()},completeLoad:function(e){var t,i,r,n=getOwn(g.shim,e)||{},o=n.exports;for(j();h.length;){if(null===(i=h.shift())[0]){if(i[0]=e,t)break;t=!0}else i[0]===e&&(t=!0);a(i)}if(l.defQueueMap={},r=getOwn(p,e),!t&&!hasProp(m,e)&&r&&!r.inited){if(!(!g.enforceDefine||o&&getGlobal(o)))return w(e)?void 0:O(makeError("nodefine","No define call for "+e,null,[e]));a([e,n.deps||[],n.exportsFn])}R()},nameToUrl:function(e,t,i){var r,n,o,a,s,u,c=getOwn(g.pkgs,e);if(c&&(e=c),u=getOwn(v,e))return l.nameToUrl(u,t,i);if(req.jsExtRegExp.test(e))a=e+(t||"");else{for(r=g.paths,o=(n=e.split("/")).length;0<o;o-=1)if(s=getOwn(r,n.slice(0,o).join("/"))){isArray(s)&&(s=s[0]),n.splice(0,o,s);break}a=n.join("/"),a=("/"===(a+=t||(/^data\:|^blob\:|\?/.test(a)||i?"":".js")).charAt(0)||a.match(/^[\w\+\.\-]+:/)?"":g.baseUrl)+a}return g.urlArgs&&!/^blob\:/.test(a)?a+g.urlArgs(e,a):a},load:function(e,t){req.load(l,e,t)},execCb:function(e,t,i,r){return t.apply(r,i)},onScriptLoad:function(e){if("load"===e.type||readyRegExp.test((e.currentTarget||e.srcElement).readyState)){interactiveScript=null;var t=s(e);l.completeLoad(t.id)}},onScriptError:function(e){var i=s(e);if(!w(i.id)){var r=[];return eachProp(p,function(e,t){0!==t.indexOf("_@r")&&each(e.depMaps,function(e){if(e.id===i.id)return r.push(t),!0})}),O(makeError("scripterror",'Script error for "'+i.id+(r.length?'", needed by: '+r.join(", "):'"'),e,[i.id]))}}}).require=l.makeRequire(),l}function getInteractiveScript(){return interactiveScript&&"interactive"===interactiveScript.readyState||eachReverse(scripts(),function(e){if("interactive"===e.readyState)return interactiveScript=e}),interactiveScript}}(this,"undefined"==typeof setTimeout?void 0:setTimeout);

<IPython.core.display.Javascript object>